In [5]:
import requests
from bs4 import BeautifulSoup as bs 
import json
import pandas as pd 
import numpy as np
import lxml
import time

In [5]:
r.url

'https://www.mangoplate.com/search/%EC%84%9C%EC%9A%B8?keyword=%EC%84%9C%EC%9A%B8&page=1'

In [166]:
from tqdm import tqdm
keyword = ["서울","경기","충청북도","충청남도","경상북도","경상남도","경상북도","강원도","전라남도","전라북도","광주광역시","인천광역시","대구광역시"
          ,"부산광역시","울산광역시"]


data_list = []

for word in tqdm(keyword):
    for i in range(1, 51):
        url = f"https://www.mangoplate.com/search/{word}?keyword={word}&page={i}"
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        try:
            r = requests.get(url, headers=header)
            r.raise_for_status()
            time.sleep(5)
        
            soup = bs(r.text, "lxml")
            for restaurant_item in soup.select(".restaurant-item"):
                restaurant_info = {}
                link = restaurant_item.select_one(".info a")['href']
                name = restaurant_item.select_one(".title").text.strip().replace("\n", "").replace(" ", "")
                address = restaurant_item.select_one(".etc").text.split("-")[0].strip()
                category = restaurant_item.select_one(".etc").text.split("-")[1].strip()
                view_count = restaurant_item.select_one(".view_count").text
                review_count = restaurant_item.select_one(".review_count").text

                restaurant_info["링크"] = link
                restaurant_info["상점명"] = name
                restaurant_info["주소"] = address
                restaurant_info["카테고리"] = category
                restaurant_info["조회수"] = view_count
                restaurant_info["리뷰"] = review_count
                data_list.append(restaurant_info)
                
        except Exception as e:
            print(f"키워드 '{word}'의 {i}번째 페이지에서 오류 발생: {e}")



 67%|██████████████████████████████████████████████████████                           | 10/15 [49:51<24:58, 299.75s/it]

키워드 '광주광역시'의 13번째 페이지에서 오류 발생: HTTPSConnectionPool(host='www.mangoplate.com', port=443): Max retries exceeded with url: /search/%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C?keyword=%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C&page=13 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002D62E0DAFA0>: Failed to resolve 'www.mangoplate.com' ([Errno 11001] getaddrinfo failed)"))


100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [1:14:27<00:00, 297.83s/it]


In [167]:
df = pd.DataFrame(data_list)

In [171]:
df

,링크,상점명,주소,카테고리,조회수,리뷰
0,/restaurants/iMRRP69qtkeO,미라이,가로수길,이자카야 / 오뎅 / 꼬치,"553,779",138
1,/restaurants/R1Nzfed1UHqa,세이류,신천/잠실,정통 일식 / 일반 일식,"190,322",43
2,/restaurants/9EVyzauilukl,시라카와,신사/압구정,이자카야 / 오뎅 / 꼬치,"352,434",76
3,/restaurants/SapvvirrPYwo,이안정,마포/공덕,회 / 스시,"26,628",22
4,/restaurants/pI112Z4JVmBn,페리지,삼성동,기타 양식,"74,056",43
...,...,...,...,...,...,...
14975,/restaurants/XGnBYklKwi,바닷가횟집,울산 동구,해산물 요리,"4,314",1
14976,/restaurants/8q36jyjNtT,개미집(옥동점),울산 남구,해산물 요리,"4,281",1
14977,/restaurants/zg5WXj8Pfd,루이스반디,울산 남구,스테이크 / 바베큐,"4,017",1
14978,/restaurants/wIzVPr91USB1,송가정횟집,울산 남구,해산물 요리,"3,812",1


In [213]:
df["ID"] = df["링크"].str.split("/restaurants/").str[1]

In [216]:
df

,링크,상점명,주소,카테고리,조회수,리뷰,ID
0,/restaurants/iMRRP69qtkeO,미라이,가로수길,이자카야 / 오뎅 / 꼬치,"553,779",138,iMRRP69qtkeO
1,/restaurants/R1Nzfed1UHqa,세이류,신천/잠실,정통 일식 / 일반 일식,"190,322",43,R1Nzfed1UHqa
2,/restaurants/9EVyzauilukl,시라카와,신사/압구정,이자카야 / 오뎅 / 꼬치,"352,434",76,9EVyzauilukl
3,/restaurants/SapvvirrPYwo,이안정,마포/공덕,회 / 스시,"26,628",22,SapvvirrPYwo
4,/restaurants/pI112Z4JVmBn,페리지,삼성동,기타 양식,"74,056",43,pI112Z4JVmBn
...,...,...,...,...,...,...,...
14975,/restaurants/XGnBYklKwi,바닷가횟집,울산 동구,해산물 요리,"4,314",1,XGnBYklKwi
14976,/restaurants/8q36jyjNtT,개미집(옥동점),울산 남구,해산물 요리,"4,281",1,8q36jyjNtT
14977,/restaurants/zg5WXj8Pfd,루이스반디,울산 남구,스테이크 / 바베큐,"4,017",1,zg5WXj8Pfd
14978,/restaurants/wIzVPr91USB1,송가정횟집,울산 남구,해산물 요리,"3,812",1,wIzVPr91USB1


In [215]:
df.to_csv("망고플레이트.csv",encoding="utf-8")

In [8]:
df = pd.read_csv("C:/Users/hi/Desktop/bigdata/망고플레이트.csv")

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import pickle


In [22]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) #웹 드라이버 버전에 맞게 다운 받고 옵션을 추가해줌 
driver.set_window_size(1920,1080)# 웹 브라우저 해상도 조절 

all_pages = []
urls = []

for index,link in enumerate(df["링크"][:5]):
    print(f"현재 {index+1}번째 내용 수집중 ... https://www.mangoplate.com{link}")
    driver.get("https://www.mangoplate.com" + link)  # 각 리뷰 페이지의 URL에 접속
    page_html = ""
    current_url = ""
    time.sleep(3)
    
    while True:
        # 현재 스크롤 위치 저장
        last_height = driver.execute_script("return document.body.scrollHeight")

        # "더 보기" 버튼을 찾아 클릭
        try:
            more_button = driver.find_element(By.CSS_SELECTOR, ".RestaurantReviewList__MoreReviewButton")
            more_button.click()
            time.sleep(5)  # 버튼 클릭 후 잠시 대기
        except:
            # 더 이상 "더 보기" 버튼을 찾을 수 없으면 루프 종료
            break

        # 스크롤이 더 이상 내려가지 않으면 루프 종료
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            time.sleep(2)
            break
        page_html = driver.page_source
        current_url = driver.current_url
        all_pages.append(page_html)
        urls.append(current_url)
    # 현재 페이지의 HTML을 all_pages 리스트에 추가

    
    # 현재 페이지의 URL을 가져와서 comment_dict와 detail_dict에 추가


현재 1번째 내용 수집중 ... /restaurants/iMRRP69qtkeO
현재 2번째 내용 수집중 ... /restaurants/R1Nzfed1UHqa
현재 3번째 내용 수집중 ... /restaurants/9EVyzauilukl
현재 4번째 내용 수집중 ... /restaurants/SapvvirrPYwo
현재 5번째 내용 수집중 ... /restaurants/pI112Z4JVmBn


In [26]:
data = []
review_data = []

for page_data,url in zip(all_pages,urls):
    soup2 = bs(page_data, "lxml")
    comment = soup2.select(".RestaurantReviewItem.RestaurantReviewList__ReviewItem")
    info = soup2.select("#restaurant_info")
    targeting = soup2.select("#dfp_targeting")
    table = soup2.select(".info>tbody")

    current_url = url  # 현재 페이지의 URL 가져오기

    for i in comment:
        comment_dict = {}
        review_text = i.select_one(".RestaurantReviewItem__ReviewText").text.strip()
        user_name = i.select_one(".RestaurantReviewItem__UserNickName").text.strip()
        rating = i.select_one(".RestaurantReviewItem__RatingText").text.strip()
        comment_dict["리뷰내용"] = review_text
        comment_dict["작성자명"] = user_name
        comment_dict["후기점수"] = rating
        comment_dict["url"] = current_url  # 현재 페이지의 URL을 추가
        review_data.append(comment_dict)

    for a, b, c in zip(info, targeting, table):
        detail_dict = {}
        location = c.select_one("tr>td").text.strip().split("지번")[0].strip()
        location2 = c.select_one("tr>td").text.strip().split("지번")[1].strip()
        number = c.select_one("tr:nth-of-type(2)>td").text.strip()
        category = [i.text.strip() for i in c.select("tr:nth-of-type(3)>td")]
        price_range = c.select_one("tr:nth-of-type(4)>td").text.strip()
        menu = [i.text for i in c.select(".Restaurant_MenuItem >span:nth-of-type(1)")]
        menu_price = [i.text for i in c.select(".Restaurant_MenuItem >span:nth-of-type(2)")]

        json_string = json.loads(a.text)
        json_district = json.loads(b.text)
        restaurant_uuid = json_string["restaurant_uuid"]
        name = json_string["name"]
        latitude = json_string["latitude"]
        longitude = json_string["longitude"]
        address = json_district["address"]
        tags = json_district["tags"]

        detail_dict["도로명주소"] = location
        detail_dict["지번주소"] = location2
        detail_dict["전화번호"] = number
        detail_dict["카테고리"] = category
        detail_dict["가격범위"] = price_range
        detail_dict["메뉴"] = menu
        detail_dict["메뉴가격"] = menu_price
        detail_dict["restaurant_uuid"] = restaurant_uuid
        detail_dict["가게명"] = name
        detail_dict["위도"] = latitude
        detail_dict["경도"] = longitude
        detail_dict["주소2"] = address
        detail_dict["태그"] = tags
        detail_dict["url"] = current_url  # 현재 페이지의 URL을 추가
        data.append(detail_dict)

In [25]:
with open('partial_crawling_results.pkl', 'wb') as f:
    pickle.dump(all_pages, f)

In [11]:
data[0]

{'도로명주소': '서울특별시 강남구 논현로153길 24 끌레르빌 1F',
 '지번주소': '서울시 강남구 신사동 557-25 끌레르빌 1F',
 '전화번호': '02-540-0934',
 '카테고리': ['이자카야 / 오뎅 / 꼬치'],
 '가격범위': '3만원-4만원',
 '메뉴': ['사시미8종 (2인)', '마구로치즈아에', '양갈비 스미비 야끼', '노도구로 시오 야끼', '규 스테이크'],
 '메뉴가격': ['41,000원', '19,000원', '23,000원', '35,000원', '41,000원'],
 'restaurant_uuid': 281547,
 '가게명': '미라이',
 '위도': '37.5210724',
 '경도': '127.0262152',
 '주소2': '서울특별시 강남구 논현로153길 24 끌레르빌 1F',
 '태그': '이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020망고플레이트인기맛집,2022망고플레이트인기맛집,가로수길,오뎅,꼬치,',
 'url': 'https://www.mangoplate.com/restaurants/iMRRP69qtkeO'}

In [27]:
df2 = pd.DataFrame(data)
df2

,도로명주소,지번주소,전화번호,카테고리,가격범위,메뉴,메뉴가격,restaurant_uuid,가게명,위도,경도,주소2,태그,url
0,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
1,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
2,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
3,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
4,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
5,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
6,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
7,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
8,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
9,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...


In [28]:
df2 = df2.drop_duplicates(subset='가게명')

In [31]:
df2

,도로명주소,지번주소,전화번호,카테고리,가격범위,메뉴,메뉴가격,restaurant_uuid,가게명,위도,경도,주소2,태그,url
0,서울특별시 강남구 논현로153길 24 끌레르빌 1F,서울시 강남구 신사동 557-25 끌레르빌 1F,02-540-0934,[이자카야 / 오뎅 / 꼬치],3만원-4만원,"[사시미8종 (2인), 마구로치즈아에, 양갈비 스미비 야끼, 노도구로 시오 야끼, ...","[41,000원, 19,000원, 23,000원, 35,000원, 41,000원]",281547,미라이,37.5210724,127.0262152,서울특별시 강남구 논현로153길 24 끌레르빌 1F,"이자카야,고등어,Hot Spot,술집추천,최자로드,핫플레이스,2018 인기,2020...",https://www.mangoplate.com/restaurants/iMRRP69...
22,서울특별시 송파구 올림픽로35가길 9 B1 B35호,서울시 송파구 신천동 11-4 B1 B35호,02-418-2288,[정통 일식 / 일반 일식],4만원 이상,"[런치스시오마카세, 디너스시오마카세, 다이나 쵸카라구치 쥰마이, 카와나카지마 토쿠베...","[60,000원, 110,000원, 65,000원, 55,000원, 30,000원]",401711,세이류,37.5174478,127.1036042,서울특별시 송파구 올림픽로35가길 9 B1 B35호,"2022망고플레이트인기맛집,스시오마카세,신천,잠실,정통 일식,일반 일식,",https://www.mangoplate.com/restaurants/R1Nzfed...
29,서울특별시 강남구 선릉로161길 7 1F,서울시 강남구 신사동 664-24 1F,070-4115-4769,[이자카야 / 오뎅 / 꼬치],2만원-3만원,"[덴푸라 10종오마카세, 네기도로 김튀김, 트러플소스 포테이토 샐러드, 우엉튀김 ...","[33,000원, 11,000원, 12,000원, 13,000원, 3,000원]",398757,시라카와,37.5270151,127.0396014,서울특별시 강남구 선릉로161길 7 1F,"이자카야,2020망고플레이트인기맛집,술집추천,2022망고플레이트인기맛집,신사,압구정...",https://www.mangoplate.com/restaurants/9EVyzau...
42,서울특별시 마포구 독막로15길 3-3 2F 101호,서울시 마포구 상수동 315-3 2F 101호,070-8615-5521,[회 / 스시],3만원-4만원,[],[],473750,이안정,37.5481121,126.9216496,서울특별시 마포구 독막로15길 3-3 2F 101호,"마포,공덕,회,스시,",https://www.mangoplate.com/restaurants/Sapvvir...
46,서울특별시 강남구 봉은사로68길 6-5 1F,서울시 강남구 삼성동 114-40 1F,기타 양식,[3만원-4만원],발렛,"[페리지 에끌래어, 봉골레, 아뇰로띠, 안다리노 윗 랍스타, 텐더로인]","[8,000원, 31,000원, 31,000원, 36,000원, 55,000원]",459822,페리지,37.511269,127.0495106,서울특별시 강남구 봉은사로68길 6-5 1F,"2022망고플레이트인기맛집,생면파스타,삼성동,기타 양식,",https://www.mangoplate.com/restaurants/pI112Z4...


In [32]:
df3 = pd.DataFrame(review_data)

In [33]:
df3["ID"] = df3["url"].str.split("/").str[4]

In [34]:
df3

,리뷰내용,작성자명,후기점수,url,ID
0,가로수길,전예원,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
1,맛있어요,전예원,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
2,언제와도 맛있는 미라이!!!고등어산도는 무조건 시켜야하구요.. (식빵 뭔지 궁금) ...,스틱스강에맹세하는맛집,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
3,"아와이 야끼소바,사바산도,우니 아마에비 호다테먹었어요.나마비루도 함께^^나마비루 부...",ㅇㅇ,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
4,- 음식리뷰 -전복내장 야끼소바 : 깔끔하게 고소하고 감칠맛이 좋음. 아주 맛있다....,최석원,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
...,...,...,...,...,...
2254,라자냐 : 잘만든 수제버거를 한 입에 먹는 맛와인리스트엔 안 보이지만 하프보틀도 있습니다,오이름,괜찮다,https://www.mangoplate.com/restaurants/pI112Z4...,pI112Z4JVmBn
2255,선정릉 주택가 골목의 언덕 위에 위치한 파스타집. 생면 파스타바의 인기에 힘입어 최...,Wandering Consultant,맛있다,https://www.mangoplate.com/restaurants/pI112Z4...,pI112Z4JVmBn
2256,"선정릉 바로 위쪽에 오픈한 엄청난 이탈리안 레스토랑 페리지. 신가영, 임홍근 부부셰...",JhY,맛있다,https://www.mangoplate.com/restaurants/pI112Z4...,pI112Z4JVmBn
2257,최근 오픈해 인기 많은 곳. 생면 파스타 좋아하는 사람은 꼭 가볼만 함! 아직 가오...,Ashley Jung,맛있다,https://www.mangoplate.com/restaurants/pI112Z4...,pI112Z4JVmBn


In [95]:
df3[df3["ID"] == "R1Nzfed1UHqa"]

,리뷰내용,작성자명,후기점수,url,ID
1488,"런치 오마카세(₩60,000)정말 말모~세이류 오랜만에 예약 성공해서 갔다왔습니다!...",Jinny67,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1489,"""세이류 디너는 나의 최애 오마카세가 되었다"" 결론적으로 디쉬 하나하나가 정말 맛있...",식존주의,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1490,협곡의 금빛섬광께서 빈자리가 났다고 알려주셔서…원래 오마카세는 있으면 먹고 아님 말...,냉청하개,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1491,세 손가락으로 집는거야 초밥은이 렇게 윗부분을 고정하고 류 현진 선수 제구하듯 세밀...,맛있으면삼행시,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1492,세이류는 세이류다.. 이름만으로 레전드수강신청보다 어렵다는 세강신청을 평일 런치에 ...,애기입맛,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
...,...,...,...,...,...
1713,"런치 오마카세(₩60,000)정말 말모~세이류 오랜만에 예약 성공해서 갔다왔습니다!...",Jinny67,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1714,"""세이류 디너는 나의 최애 오마카세가 되었다"" 결론적으로 디쉬 하나하나가 정말 맛있...",식존주의,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1715,협곡의 금빛섬광께서 빈자리가 났다고 알려주셔서…원래 오마카세는 있으면 먹고 아님 말...,냉청하개,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa
1716,세 손가락으로 집는거야 초밥은이 렇게 윗부분을 고정하고 류 현진 선수 제구하듯 세밀...,맛있으면삼행시,맛있다,https://www.mangoplate.com/restaurants/R1Nzfed...,R1Nzfed1UHqa


In [35]:
df

,Unnamed: 0,링크,상점명,주소,카테고리,조회수,리뷰,ID
0,0,/restaurants/iMRRP69qtkeO,미라이,가로수길,이자카야 / 오뎅 / 꼬치,"553,779",138,iMRRP69qtkeO
1,1,/restaurants/R1Nzfed1UHqa,세이류,신천/잠실,정통 일식 / 일반 일식,"190,322",43,R1Nzfed1UHqa
2,2,/restaurants/9EVyzauilukl,시라카와,신사/압구정,이자카야 / 오뎅 / 꼬치,"352,434",76,9EVyzauilukl
3,3,/restaurants/SapvvirrPYwo,이안정,마포/공덕,회 / 스시,"26,628",22,SapvvirrPYwo
4,4,/restaurants/pI112Z4JVmBn,페리지,삼성동,기타 양식,"74,056",43,pI112Z4JVmBn
...,...,...,...,...,...,...,...,...
14975,14975,/restaurants/XGnBYklKwi,바닷가횟집,울산 동구,해산물 요리,"4,314",1,XGnBYklKwi
14976,14976,/restaurants/8q36jyjNtT,개미집(옥동점),울산 남구,해산물 요리,"4,281",1,8q36jyjNtT
14977,14977,/restaurants/zg5WXj8Pfd,루이스반디,울산 남구,스테이크 / 바베큐,"4,017",1,zg5WXj8Pfd
14978,14978,/restaurants/wIzVPr91USB1,송가정횟집,울산 남구,해산물 요리,"3,812",1,wIzVPr91USB1
